# 1. Nativní C rozšíření pro Python
Python C API (Application Programming Interface) umožňuje napsat modul v C, který se z Pythonu používá stejně jako běžný importovatelný modul.

V této části si projdeme minimální „povinné“ stavební bloky: parsování argumentů, registraci metod, definici modulu a inicializaci.


## 1.1 Parsování argumentů (`PyArg_ParseTuple`)
Typická signatura exportované funkce je:

```c
static PyObject *mvp_func_py(PyObject *self, PyObject *args)
```

Co znamenají parametry:
- `PyObject *self`: objekt modulu nebo instance (podle typu funkce),
- `PyObject *args`: n-tice argumentů předaná z Pythonu,
- návrat `PyObject *`: vždy vrací Python objekt, nebo `NULL` při chybě.

`PyArg_ParseTuple` čte Python argumenty podle formátovacího řetězce. V této ukázce je nejdřív přečteme jako `numpy.ndarray` a potom ještě ručně ověříme:

- `dtype` (`float32`),
- počet dimenzí,
- C-contiguous layout,
- kompatibilní tvary `matrix`, `vector`, `result`.


## 1.2 Registrace metod (`PyMethodDef`)
Pole `PyMethodDef` mapuje název Python funkce na implementaci v C:

```c
static PyMethodDef MatrixVectorMultiplyMethods[] = {
    {"mvp_func_py", mvp_func_py, METH_VARARGS, "..."},
    {NULL, NULL, 0, NULL}
};
```

Význam položek:
- jméno funkce viditelné v Pythonu,
- ukazatel na C implementaci,
- flag volací konvence (`METH_VARARGS` = argumenty přijdou v `args` n-tici),
- dokumentační řetězec.

Poslední položka s `NULL` je ukončovací značka pole.


## 1.3 Definice modulu (`PyModuleDef`)
`PyModuleDef` popisuje modul: jméno, dokumentaci, stav modulu a seznam metod.

Důležité položky:
- `PyModuleDef_HEAD_INIT`: povinná inicializace hlavičky struktury,
- `-1`: modul nepoužívá per-interpreter stav,
- pole metod: seznam funkcí exportovaných do Pythonu.


## 1.4 Inicializace modulu (`PyInit_*`)
Inicializační funkce:

- vytvoří modul přes `PyModule_Create`,
- zavolá inicializaci NumPy C API (`_import_array()`),
- vrátí hotový modul, který Python načte přes `import`.

`PyMODINIT_FUNC` nastaví správnou exportní signaturu symbolu pro danou platformu.


In [ ]:
%%writefile mvc_mylib.c
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION

#include <Python.h>
#include <numpy/arrayobject.h>

#include "matrix_vector_multiply.h"

static int validate_inputs(PyArrayObject *matrix, PyArrayObject *vector, PyArrayObject *result) {
    if (PyArray_TYPE(matrix) != NPY_FLOAT32 || PyArray_NDIM(matrix) != 2 || !PyArray_IS_C_CONTIGUOUS(matrix)) {
        PyErr_SetString(PyExc_TypeError, "matrix musí být C-contiguous ndarray typu float32 se 2 dimenzemi.");
        return 0;
    }
    if (PyArray_TYPE(vector) != NPY_FLOAT32 || PyArray_NDIM(vector) != 1 || !PyArray_IS_C_CONTIGUOUS(vector)) {
        PyErr_SetString(PyExc_TypeError, "vector musí být C-contiguous ndarray typu float32 s 1 dimenzí.");
        return 0;
    }
    if (PyArray_TYPE(result) != NPY_FLOAT32 || PyArray_NDIM(result) != 1 || !PyArray_IS_C_CONTIGUOUS(result)) {
        PyErr_SetString(PyExc_TypeError, "result musí být C-contiguous ndarray typu float32 s 1 dimenzí.");
        return 0;
    }
    if (!PyArray_ISWRITEABLE(result)) {
        PyErr_SetString(PyExc_TypeError, "result musí být zapisovatelný ndarray.");
        return 0;
    }

    if (PyArray_DIM(vector, 0) != PyArray_DIM(matrix, 1)) {
        PyErr_SetString(PyExc_ValueError, "Délka vector musí odpovídat počtu sloupců matrix.");
        return 0;
    }
    if (PyArray_DIM(result, 0) != PyArray_DIM(matrix, 0)) {
        PyErr_SetString(PyExc_ValueError, "Délka result musí odpovídat počtu řádků matrix.");
        return 0;
    }

    return 1;
}

static PyObject *mvp_func_py(PyObject *self, PyObject *args) {
    PyArrayObject *matrix = NULL;
    PyArrayObject *vector = NULL;
    PyArrayObject *result = NULL;

    if (!PyArg_ParseTuple(
            args,
            "O!O!O!",
            &PyArray_Type,
            &matrix,
            &PyArray_Type,
            &vector,
            &PyArray_Type,
            &result)) {
        return NULL;
    }

    if (!validate_inputs(matrix, vector, result)) {
        return NULL;
    }

    matrix_vector_multiply(
        (const float *)PyArray_DATA(matrix),
        (const float *)PyArray_DATA(vector),
        (float *)PyArray_DATA(result),
        (size_t)PyArray_DIM(matrix, 0),
        (size_t)PyArray_DIM(matrix, 1));

    Py_RETURN_NONE;
}

static PyMethodDef MatrixVectorMultiplyMethods[] = {
    {"mvp_func_py", mvp_func_py, METH_VARARGS, "Spočítá matrix @ vector do prealokovaného result."},
    {NULL, NULL, 0, NULL},
};

static struct PyModuleDef matrix_vector_multiply_module = {
    PyModuleDef_HEAD_INIT,
    "mvc_mylib",
    "Nativní C modul pro násobení matice a vektoru.",
    -1,
    MatrixVectorMultiplyMethods,
};

PyMODINIT_FUNC PyInit_mvc_mylib(void) {
    PyObject *module = PyModule_Create(&matrix_vector_multiply_module);
    if (module == NULL) {
        return NULL;
    }

    if (_import_array() < 0) {
        Py_DECREF(module);
        return NULL;
    }

    return module;
}


In [ ]:
%%writefile setup.py
from setuptools import Extension, setup
import numpy as np

matrix_vector_multiply_module = Extension(
    "mvc_mylib",
    sources=["mvc_mylib.c", "matrix_vector_multiply.c"],
    include_dirs=[np.get_include(), "."],
    extra_compile_args=["-O3", "-std=c11"],
)

setup(
    name="mvc_mylib",
    version="1.0",
    description="A C extension for multiplying a matrix and a vector",
    ext_modules=[matrix_vector_multiply_module],
)


In [ ]:
import os
import sys

os.environ["PATH"] = f"{os.path.dirname(sys.executable)}:{os.environ['PATH']}"


In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import numpy as np
import mvc_mylib

matrix = np.ascontiguousarray(
    [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]],
    dtype=np.float32,
)
vector = np.ascontiguousarray([2, 1, 3, 0], dtype=np.float32)
result = np.zeros(3, dtype=np.float32)

mvc_mylib.mvp_func_py(matrix, vector, result)

print("C extension:")
print(result)


In [ ]:
matrix @ vector